You can use this notebook to visulalize the stress-energy tensor in (t,r) (both classical and the quantum contribution). You need to perform a semiclassical simulation (with or without backreaction) and store "data", and "bilinears". It assumes that both data-types are saved at the same timestpes. If not, you need to modify appropriately.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpecialFunctions
using FunctionZeros
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.4_b0.0_c4.0_rmax15.0_tmax7.0_cfl0.0625_sigma0.0_overMp2_25.132741228718345_reg_true_backreact_false_mPV100.0_dk_denom_15_kmax10.0_lmax30.0"
your_dir = dir*par

In [ ]:
# set the number of grid points you used
D = 3
Nr = 128*2^D + 3

# make directory to save figures
out_dir = "./stress-energy_tensor_tr/"*par*"/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames)           = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");
(its_bln, all_filenames_bln)   = list_h5_files(your_dir*"/data_$(Nr)", prefix="bilinears_");

# classical data
println("length(its)=",length(its))
println("its",its)

# bilinears
println("length(its_bln)=",length(its_bln))
println("its_bln=",its_bln)

The cell below builds the components of the stress energy tensor, both purely classical and semiclassical (classical + quantum), as functions of (t,r). It also constructs the outgoing energy in the null direction (TUU).

In [ ]:
# convention
hbar = 1
c = 1

# how many timesteps to consider
ti_max = length(its_bln)

# initiate lists to save (t,r) data
U_tr = zeros(ti_max, length(r));
V_tr = zeros(ti_max, length(r));
f_tr = zeros(ti_max, length(r));
g_tr = zeros(ti_max, length(r));

Φ_tr = zeros(ti_max, length(r));
Π_tr = zeros(ti_max, length(r));
Ψ_tr = zeros(ti_max, length(r));

α_tr  = zeros(ti_max, length(r));
A_tr  = zeros(ti_max, length(r));
B_tr  = zeros(ti_max, length(r));
KB_tr = zeros(ti_max, length(r));

# classical
Ttt_tr = zeros(ti_max, length(r));
Ttr_tr = zeros(ti_max, length(r));
Trr_tr = zeros(ti_max, length(r));
#quantum <T_{ab}(t,r)>
Ttt_tr_q = zeros(ti_max, length(r));
Ttr_tr_q = zeros(ti_max, length(r));
Trr_tr_q = zeros(ti_max, length(r));

# loop over time to fill in the above lists
for i in 1:ti_max
    
    # if you have not saved data and bilinears on the same timesteps
    # you need to change it below, accordingly, as well as it_str in bilin below
    it = its[i]
    it_str  = lpad(it, 4, "0")
    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]

    U   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,17]
    V   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,18]
    f   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,15]
    g   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,16]

    Φ   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,1]
    Π   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,2]
    Ψ   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,3]

    α   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,11]
    A   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,4]
    B   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,5]
    KB  =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,9]

    bilin  = h5read(your_dir*"/data_$(Nr)/bilinears_$(it_str).h5","bilinears")

    U_tr[i,:] = U
    V_tr[i,:] = V
    f_tr[i,:] = f
    g_tr[i,:] = g

    Φ_tr[i,:] = Φ
    Π_tr[i,:] = Π
    Ψ_tr[i,:] = Ψ

    α_tr[i,:]  = α
    A_tr[i,:]  = A
    B_tr[i,:]  = B
    KB_tr[i,:] = KB

    # <Π^2> = Π^2 + (AB^2/α^2)*hbar*c^2*(bilin[2])/(4π)
    Π2_q = @. Π[:]^2 + hbar*c^2*real.(bilin[:,2])*A[:]*B[:]^2/(α[:]^2*4*π)
    # <Ψ^2> = Ψ^2 + hbar^c^2*bilin[3]/(4π)
    Ψ2_q = @. Ψ[:]^2 + hbar*c^2*real.(bilin[:,3])/(4*π)
    # <Π Ψ> = Π*Ψ + bilin[4]*hbar*c^2*B*A^(1/2)/(α*4*π)
    ΠΨ_q = @. Π[:]*Ψ[:] + real.(bilin[:,4])*hbar*c^2*B[:]*A[:]^0.5/(α[:]*4*π)
    # <(\partial_θ Φ)^2> = hbar*c^2*bilin[5]/(4*π)
    Φθ2_q = real.(bilin[:,5])*hbar*c^2/(4*π)

    # <T_tt> = α^2/(2A) [<Π^2>/B^2 + <Ψ^2>] + α^2/(B r^2) * <(\partial_θ Φ)^2>
    Ttt_tr_q[i,:] = @. α[:]^2/(2*A[:])*(Π2_q[:]/B[:]^2 + Ψ2_q[:]) + α[:]^2/(B[:]*r[:]^2)*Φθ2_q[:]
    # <T_tr> = α <Π Ψ> /(B A^0.5)
    Ttr_tr_q[i,:] = @. α[:]*ΠΨ_q[:]/(B[:]*A[:]^0.5)
    # <T_rr> = 0.5*[<Π^2>/B^2 + <Ψ^2>] - A/(B r^2) * <(\partial_θ Φ)^2>
    Trr_tr_q[i,:] = @. 0.5*(Π2_q[:]/B[:]^2 + Ψ2_q[:]) - A[:]/(B[:]*r[:]^2)*Φθ2_q[:]

    #classic
    # Π^2
    Π2 = @. Π[:]^2
    # Ψ^2 
    Ψ2 = @. Ψ[:]^2
    # Π*Ψ 
    ΠΨ = @. Π[:]*Ψ[:]
    # T_tt = α^2/(2A) [Π^2/B^2 + Ψ^2]
    Ttt_tr[i,:] = @. α[:]^2/(2*A[:])*(Π2[:]/B[:]^2 + Ψ2[:])
    # T_tr = α Π Ψ /(B A^0.5)
    Ttr_tr[i,:] = @. α[:]*ΠΨ[:]/(B[:]*A[:]^0.5)
    # T_rr = 0.5*[Π^2/B^2 + Ψ^2]
    Trr_tr[i,:] = @. 0.5*(Π2[:]/B[:]^2 + Ψ2[:])
    
end

dtdU_tr = 0.5./(α_tr.*f_tr);
# Sometimes the quantity inside sqrt can get negative, which is a problem.
# There is a cell below that checks this if needed (uncomment)
drdU_tr = - 0.5./sqrt.(A_tr.*f_tr); 

# T_UU = dt/dU*dt/dU*T_tt + 2*dt/dU*dr/dU*T_tr + dr/dU*dr/dU*T_rr

TUU_tr = @. Ttt_tr*dtdU_tr^2 + 2.0*Ttr_tr*dtdU_tr*drdU_tr + Trr_tr*drdU_tr^2;
TUU_tr_q = @. Ttt_tr_q*dtdU_tr^2 + 2.0*Ttr_tr_q*dtdU_tr*drdU_tr + Trr_tr_q*drdU_tr^2;

# you can uncomment below to construct: T^VV = T_UU/g_UV^2 
#g_UV = -1/(2*f*g)
#gUV_tr = @. -1/(2*f_tr*g_tr);
#TVVup_tr = @. TUU_tr/(gUV_tr^2);

# construct the t_list and list of position of the apparent horizon, in time
# set manually dt (64 is every how many steps you save data and 16 is from the CFL condition
dt = 64*dr/16
tlist = zeros(ti_max)
AH_t  = zeros(ti_max)
for i in 1:ti_max-1
    tlist[i+1] = tlist[i]+dt
    AH_t[i] = find_AH(r, B_tr[i,:], A_tr[i,:], KB_tr[i,:])
end
AH_t[end] = find_AH(r, B_tr[end,:], A_tr[end,:], KB_tr[end,:]);
#println(tlist)
#println(AH_t)

In [ ]:
# uncomment if you have problem with cell above
# then you also need to tune the r interval above (now it uses the whole r domain)

#for i in 1:ti_max
#    for j in 1:Nr
#        if (A_tr.*f_tr)[i,j] <0
#            println("(i,j) = ", (i,j))
#        end
#    end
#end

As a reference, you can plot the scalar field in (t,r):

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], Φ_tr[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"\Phi(t,r)",
    xlabel = "r", ylabel="time",
    frame=true, xlim=(0,8), ylim=(0,6.9))

p = scatter!((AH_t, tlist),label="AH(t,r)",  markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save it
#savefig(p, out_dir*"/Phi_tr.pdf")

Plot various energy-tensor components in (t,r):

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], Ttt_tr[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"T_{tt}(t,r)",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save it
#savefig(p, out_dir*"/Ttt_tr.pdf")

In [ ]:
rmin = 10
rmax = Nr

contourf(r[rmin:rmax], tlist[:], Ttt_tr_q[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"\langle T_{tt}(t,r) \rangle",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save 
#savefig(p, out_dir*"/Ttt_tr_q.pdf")

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], Ttr_tr[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"T_{tr}(t,r)",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/Ttr_tr.pdf")

In [ ]:
rmin = 3
rmax = Nr

contourf(r[rmin:rmax], tlist[:], Ttr_tr_q[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"\langle T_{tr}(t,r) \rangle",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/Ttr_tr_q.pdf")

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], Trr_tr[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"T_{rr}(t,r)",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/Trr_tr.pdf")

In [ ]:
rmin = 10
rmax = Nr

contourf(r[rmin:rmax], tlist[:], Trr_tr_q[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"\langle T_{rr}(t,r) \rangle",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/Trr_tr_q.pdf")

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], TUU_tr[:,rmin:rmax],
    color=:thermal, wsize = (800,700), title = L"T_{UU}(t,r)",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p=scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/TUU_tr.pdf")

In [ ]:
rmin = 3
rmax = 550#Nr

contourf(r[rmin:rmax], tlist[:], (TUU_tr_q[:,rmin:rmax]),
    color=:thermal, wsize = (800,700), title = L"\langle T_{UU}(t,r) \rangle",
    xlabel = "r", ylabel="time", frame=true,
    xlim=(0,8), ylim=(0,6.9))
p = scatter!((AH_t, tlist),label="AH(t,r)", markerstrokewidth=0, mc=:green, ms=4, legend=:bottomright)
# uncomment if you want to save
#savefig(p, out_dir*"/TUU_tr_q.pdf")